In [1]:
### código para descargar los pdfs de los enlaces de cada contrato en la página de contratación de México. Descarga todos los archivos en pdf y crea uan carpeta con el nombre del expediente apra guardarlos################

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#from highlight_sel_element import highlight
from selenium.webdriver.common.keys import Keys
import pandas as pd
import os
import re
import time
#import PyPDF2
import collections
import requests
import threading
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException

############ codigo para descargar elementos faltantes en anterior iteración ################

df = pd.read_excel('/Users/datos-lc/Documents/Colecciones ongoing/SEDENA/Contratos Guardia Nacional_2022.xlsx')
carpeta_descargas = os.path.expanduser('/Users/datos-lc/Downloads/')

no_descargados = []  # Lista para almacenar los posibles errores
descargados = []  # Lista para almacenar los contratos descargados

### for que usa dos parámetros del dataframe. Para cada una de esas columnas hace las búsquedas y descargas.
for columna1, columna2 in zip(df['Código del expediente'], df['Dirección del anuncio']):
   
    # Número del expediente que aparece en la columna del excel, para comprobar después que sí sea ese el contrato descargado
    expediente_excel = str(columna1)

    # Navegar en este enlace
    driver = webdriver.Chrome(columna2)
    driver.get(columna2)
    
    # Obtener el titulo con información del expediente y nombre del contrato que aparece en la página, sirve para nombrar la carpeta con los PDFs
    titulo = driver.find_element(By.XPATH, "//div[@class='subtitle_01 maintitle']")
    nombre_carpeta_contrato = titulo.text.strip()

    # Ruta completa de la carpeta para los archivos del contrato
    ruta_contrato = os.path.join(f'/Users/datos-lc/Documents/Colecciones ongoing/SEDENA/2022/', nombre_carpeta_contrato)
    
#Si la carpeta con el nombre del contrato existe, ya tiene contenido

    if os.path.exists(ruta_contrato):
        
        print(f"la carpeta{nombre_carpeta_contrato} ya tiene contenido")
        print(" ------------------------------------------------------")
        driver.quit()
#Si no existe no ha sido descargada y procede a crear la carpeta y buscar los pdf para descargarlos.       
    else:
        #Crea la carpeta
        os.makedirs(ruta_contrato)
        print(f"la carpeta{nombre_carpeta_contrato} NO EXISTEEEEEEEEEEE") 
        print(" ------------------------------------------------------")
       # Leer el número del expediente que aparece en la página web que estamos navegando
        aux = driver.find_element(By.XPATH, "//div[@class='form_container'][1]/ul/li/div[@class='form_answer']")
        expediente_pagina = aux.text.strip()

        #Crear la lista con los elementos descargables
        pdf = driver.find_elements(By.XPATH, "//a[@class='attachTaglink']")
        cantidad_pdf = len(pdf)
        print(cantidad_pdf)

#Cada archivo en pdf se le da click para descargar, si no cargó la página guarda la información que puede y continúa
        for archivo in pdf:
            try:
                archivo.click()
                time.sleep(11)
            except StaleElementReferenceException:
                print("Elemento obsoleto. Se omitirá y continuará con el siguiente.")
                print(nombre_carpeta_contrato)
                print(columna2)
                no_descargados.append([nombre_carpeta_contrato, cantidad_pdf, expediente_excel, expediente_pagina, columna2])
                continue
#Hace una lista de los archivos descargados DEBE ESTAR VACIA LA CARPETA DE DESCARGAS ANTES DE INICIAR EL CODIGO
        files = os.listdir(carpeta_descargas)

        if len(files)> 0:

            for descargado in files:
                ruta_vieja = os.path.join(carpeta_descargas, descargado)
                ruta_nueva = os.path.join(ruta_contrato, descargado) 
                os.rename(ruta_vieja, ruta_nueva)
                descargados.append([nombre_carpeta_contrato, cantidad_pdf, expediente_excel, expediente_pagina, descargado, columna2])

        else:
            print("No se encontraron archivos en la carpeta de descargas.")
            no_descargados.append([nombre_carpeta_contrato, cantidad_pdf, expediente_excel, expediente_pagina])

        driver.quit()


df_no_descargados = pd.DataFrame(no_descargados, columns=["Titulo_contrato", "Nro_pdfs_en_contrato", "Expediente_excel", "Expediente_pagina"])
df_descargados = pd.DataFrame(descargados, columns=["Titulo_contrato", "Nro_pdfs_en_contrato", "Expediente_excel", "Expediente_pagina", "Nombre_archivo_descargado", "URL" ])
df_descargados.to_csv('/Users/datos-lc/Documents/Colecciones ongoing/SEDENA/2022/descargados.csv', index=False)
df_no_descargados.to_csv('/Users/datos-lc/Documents/Colecciones ongoing/SEDENA/2022/no-descargados.csv', index=False)

<ipython-input-1-6dada79cef10>:38: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(columna2)


la carpetaExpediente 2401772 - Servicio de Testigo Social para intervenir en el procedimiento de Licitación Pú NO EXISTEEEEEEEEEEE
 ------------------------------------------------------
2
la carpetaExpediente 2410081 - Servicio de suministro de alimentos para los empleados de la Guardia Nacional ya tiene contenido
 ------------------------------------------------------
la carpetaExpediente 2412743 - CONTRATACIÓN DEL SERVICIO DE ASEGURAMIENTO DE LOS BIENES PATRIMONIALES PROPIEDAD NO EXISTEEEEEEEEEEE
 ------------------------------------------------------
1
la carpetaExpediente 2427902 - Servicio de mantenimiento preventivo y correctivo a elevadores de la marca OTIS NO EXISTEEEEEEEEEEE
 ------------------------------------------------------
2
la carpetaExpediente 2429786 - Servicio de Mantenimiento Preventivo y Correctivo de las Salas de Conferencias ya tiene contenido
 ------------------------------------------------------
la carpetaExpediente 2434742 - SERVICIO DE RECOLECCIÓN DE RESID

FileNotFoundError: [Errno 2] No such file or directory: '/Users/datos-lc/Downloads/.com.google.Chrome.bTlmPD' -> '/Users/datos-lc/Documents/Colecciones ongoing/SEDENA/2022/Expediente 2525992 - Servicio de mantenimiento integral preventivo, correctivo y respaldo técnico de unidades portátiles de detección de narcóticos y sustancias controladas./.com.google.Chrome.bTlmPD'